Conversion required:
* In the forward processing below, the eigenvector is 8-bit and the input is ternary but represented in 8-bits. But the result may be bit_depth_max\*L - actually as the eigenvector is normalised this will be int(1/sqrt(L))\*127)\*L.
* In the backward processing, processed_r_back is ternary (but stored as 8-bit) and eigenvector is 8-bit as above. The result is either -1\*eigenvector, 0 or eigenvector.
* We can likely implement all casting control in the project method.

In [1]:
import numpy as np

In [2]:
from src.var_processor.covariance8bit import CovarianceUnit
from src.var_processor.power_iterator8bit import PowerIterator
from src.var_processor.pb_threshold import ternary_pbt

def project(vec_1, vec_2):
    """Project input using eigenvector.

    Args:
        vec1: 1D numpy array.
        vec2: 1D numpy array.
    """
    return np.dot(vec_1, vec_2)


class VPU:
    """Variance processing unit."""

    def __init__(self, size):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input;
        """
        self.cu = CovarianceUnit(size)
        self.pi = PowerIterator(size)
        self.size = size

    def forward_processing(self, forward_data):
        """Process data to apply to forward input data."""
        return forward_data

    def pred_input_processing(self, pred_inputs):
        """Process data to apply to output predicted inputs."""
        return pred_inputs

    def r_forward_processing(self, r_forward):
        """Process data to apply to output r_forward value."""
        return r_forward

    def r_backward_processing(self, r_backward):
        """Process data to apply to output r_forward value."""
        return r_backward

    def iterate(self, forward_data, r_backward):
        """Iterate through one discrete timestep.

        Args:
            forward_data: data for feedforward transformation
                1D numpy array of length self.size.
            r_backward: scalar value indicating a prediction of r.

        Returns:
            pred_inputs: 1D array containing the predicted input
            r: scalar feature detection output

        """
        r_forward = self.forward(forward_data)
        pred_inputs = self.backward(r_backward)
        return r_forward, pred_inputs

    def forward(self, forward_data):
        """Forward pass to generate cause - r.

        Args:
            forward_data: 1D numpy array of length self.size.
            This is the residual data rather than the original data.
        Returns:
            r_forward: scalar feature detection output

        """
        # Perform optional pre-processing
        processed_data = self.forward_processing(forward_data)
        # Project
        r_forward = project(self.pi.eigenvector.T, processed_data)
        # Perform optional post-processing
        processed_output = self.r_forward_processing(r_forward)
        return processed_output

    def backward(self, r_backward):
        """Backward pass to generate predicted inputs.

        The predicted inputs are the original not residual inputs.

        Args:
            r_backward: scalar cause feedback.
        Returns:
            pred_inputs: numpy array of predicted inputs of size - size.

        """
        # Perform optional pre-processing
        processed_r_back = self.r_backward_processing(r_backward)
        # Use item to convert r to scalar
        pred_inputs = project(processed_r_back.item(), self.pi.eigenvector)
        # Perform optional post-processing
        processed_output = self.pred_input_processing(pred_inputs)
        return processed_output

    def update_cov(self, input_data, power_iterate=False):
        """Update the covariance matrix.

        Use this to bed in the covariance.

        Args:
            input_data: 1D numpy array of length self.size.
            This is the original rather than residual data.
        """
        self.cu.update_cov(input_data)
        if power_iterate:
            cov = self.cu.covariance
            # Power iterate
            self.pi.load_covariance(cov)
            self.pi.iterate()

    def reset(self):
        """Reset and clear."""
        self.__init__(self.size)

### Perform Original VPU Tests on Above 

We'll now get integer values.

In [3]:
from src.tests.test_vpu import rand_same, rand_diff, rand_opposite

In [4]:
vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (1/np.sqrt(2))*127)

[[70 70]
 [70 70]] [[0.5511811 0.5511811]
 [0.5511811 0.5511811]]
[[-89]
 [-89]] 89.80256121069152


In [5]:
ones = np.ones(shape=(2,1), dtype=np.int8); ones

array([[1],
       [1]], dtype=int8)

In [6]:
vpu.forward(ones)

array([[78]], dtype=int8)

Overflow.

In [7]:
processed_data = ones*(np.sqrt(2)/2)
project(vpu.pi.eigenvector.T, processed_data).astype(np.int8)

array([[-125]], dtype=int8)

In [8]:
odd = np.asarray([[1], [0]]); odd

processed_data = odd*(np.sqrt(2)/2)
project(vpu.pi.eigenvector.T, processed_data).astype(np.int8)

array([[-62]], dtype=int8)

Scaling works.

In [9]:
vpu.pi.eigenvalue

array([140], dtype=uint16)

In [10]:
sample_1 = np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue)); sample_1

array([-1053.0623, -1053.0623], dtype=float32)

In [11]:
sample_1 = np.dot(vpu.pi.eigenvector/127, np.sqrt(vpu.pi.eigenvalue/127)); sample_1

array([-0.73578079, -0.73578079])

In [12]:
vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(negative=True), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (1/np.sqrt(2))*127)

[[84 84]
 [84 84]] [[0.66141732 0.66141732]
 [0.66141732 0.66141732]]
[[-89]
 [-89]] 89.80256121069152


In [13]:
vpu.pi.eigenvalue

array([168], dtype=uint16)

In [14]:
sample_1 = np.dot(vpu.pi.eigenvector/127, np.sqrt(vpu.pi.eigenvalue/127)); sample_1

array([-0.80600747, -0.80600747])

In [23]:
size = 4
vpu = VPU(size)
for _ in range(0, 1000):
    vpu.update_cov(rand_same(negative=True, size=size), power_iterate=True)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance/127)
print(vpu.pi.eigenvector, (np.sqrt(4)/4)*127)

[[77 77 77 77]
 [77 77 77 77]
 [77 77 77 77]
 [77 77 77 77]] [[0.60629921 0.60629921 0.60629921 0.60629921]
 [0.60629921 0.60629921 0.60629921 0.60629921]
 [0.60629921 0.60629921 0.60629921 0.60629921]
 [0.60629921 0.60629921 0.60629921 0.60629921]]
[[63]
 [63]
 [63]
 [63]] 63.5


In [24]:
vpu.pi.eigenvector

array([[63],
       [63],
       [63],
       [63]], dtype=int8)

In [25]:
size = 4
ones = np.ones(shape=(size,1), dtype=np.int8); print(ones)
processed_data = ones*(np.sqrt(size)/size); print(processed_data)
print(project(vpu.pi.eigenvector.T, processed_data))
print(project(vpu.pi.eigenvector.T, processed_data).astype(np.int8))

[[1]
 [1]
 [1]
 [1]]
[[0.5]
 [0.5]
 [0.5]
 [0.5]]
[[126.]]
[[126]]


In [26]:
size = 4
neg_ones = -1*np.ones(shape=(size,1), dtype=np.int8); print(neg_ones)
processed_data = neg_ones*(np.sqrt(size)/size); print(processed_data)
print(project(vpu.pi.eigenvector.T, processed_data))
print(project(vpu.pi.eigenvector.T, processed_data).astype(np.int8))

[[-1]
 [-1]
 [-1]
 [-1]]
[[-0.5]
 [-0.5]
 [-0.5]
 [-0.5]]
[[-126.]]
[[-126]]


Ah - so this should be reversed sign and we are getting overflow. Our eigenvectors are one integer too big at -64 - should be -63. On the positive side we are still fine and have 63.

We can now PBT with max value = 126.

In [ ]:
-127//10.9, -127/10.9, 127//10.9, 127/10.9

Ah that is why - integer rounds down for negative numbers. We need to adjust our normalise function. **FIXED** (By taking the sign out.

In [ ]:
class BinaryVPU(VPU):
    """Let's update our functions modularly."""

    def __init__(self, size):
        """Adapted Init."""
        super(BinaryVPU, self).__init__(size)
        # Calculate scale factor here to save time later
        self.scale_forward = np.sqrt(self.size)/self.size
        self.scale_backward = self.size/np.sqrt(self.size)

    def r_forward_processing(self, r_forward):
        """Scale r to -1 to 1 and PBT."""
        # Scale to ternary
        scaled_r = r_forward*self.scale_forward
        # Threshold r
        binary_values = ternary_pbt(scaled_r)
        return binary_values

    def r_backward_processing(self, r_backward):
        """Rescale r to -L/sqrt(L) to L/sqrt(L) and PBT."""
        # Scale to ternary
        scaled_r = r_backward*self.scale_backward
        return scaled_r

    def pred_input_processing(self, pred_inputs):
        """Apply PBT to get outputs in range -1, 0, 1."""
        binary_values = ternary_pbt(pred_inputs)
        return binary_values